In [1]:
import time
import sys
import os
from collections import OrderedDict
from importlib import reload

import h5py
import numpy as np
from matplotlib import pyplot as plt

%matplotlib notebook

In [2]:
import qcodes
qc = qcodes

# Basic configuration

In [32]:
%run -n init_vars.py
%run -n init_instruments.py
%run -n init_defs.py

station = qcodes.Station(key, ivvi, RF, LO, alazar, fg, awg)

2017-12-30 20:18:52,190 : measurement : INFO : Logger set up!
C:\Users\Administrator\Code\Qcodes\qcodes\instrument\parameter.py:514: UserWarning: set_step is deprecated use step property as in `inst.step = stepvalue` instead
  "set_step is deprecated use step property as in `inst.step = "
C:\Users\Administrator\Code\Qcodes\qcodes\instrument\parameter.py:526: UserWarning: set_delay is deprecated use inter_delay or post_delay property as in `inst.inter_delay = delayvalue` instead
  "set_delay is deprecated use inter_delay or post_delay property "


Instrument ivvi already exists.
Instrument key1 already exists.
Instrument RF already exists.
Instrument LO already exists.
Instrument alazar already exists.
Instrument awg already exists.
Instrument fg already exists.


In [36]:
ivvi.dac1.set_delay(0.001)
ivvi.dac1.set_step(20)
ivvi.dac6.set_delay(0.001)
ivvi.dac6.set_step(5)
ivvi.dac8.set_delay(0.001)
ivvi.dac8.set_step(5)
ivvi.dac12.set_delay(0.001)
ivvi.dac12.set_step(5)
ivvi.dac4.set_delay(0.001)
ivvi.dac4.set_step(5)
ivvi.dac7.set_delay(0.001)
ivvi.dac7.set_step(5)

C:\Users\Administrator\Code\Qcodes\qcodes\instrument\parameter.py:526: UserWarning: set_delay is deprecated use inter_delay or post_delay property as in `inst.inter_delay = delayvalue` instead
  "set_delay is deprecated use inter_delay or post_delay property "
C:\Users\Administrator\Code\Qcodes\qcodes\instrument\parameter.py:514: UserWarning: set_step is deprecated use step property as in `inst.step = stepvalue` instead
  "set_step is deprecated use step property as in `inst.step = "


# Experiments

## Test: alazar time trace

In [ ]:
%run -i snippets/alazar_timetrace.py

## RF conductance on the bias resonator

In [37]:
station.ivvi.dac4(-905)
station.ivvi.dac5(-615)
station.ivvi.dac6(-850)

In [42]:
ivvi.dac7(-2000)

In [44]:
# make sure we trigger externally
namespace.ats_settings['trigger_source1'] = 'EXTERNAL'

# set sampling rate of the alazar
namespace.ats_settings['sample_rate'] = int(1e8)

# awg as low sampling as possible
namespace.awg_settings['sampling_rate'] = int(1e7)

# limit awg output
namespace.awg_settings['channel_2']['analog_amplitude'] = 0.1

# set the funcgen frequency
# TODO: should be done automatically
station.fg.ch1_frequency(40)

# create measurement class and run
m = AWGGateSweep(station, namespace)
m.awg_chan(2)
m.ramp_pts(200)
m.max_val(0.025)
m.ramp_down_len(20e-6)
m.ats_int_time(50e-6)
m.navgs(1)
m.IF(8e6)
m.sweep = [
    (m.station.ivvi.dac7, np.linspace(-2000, 2000, 101))
]
m.run()

2017-12-30 20:35:06,008 : measurement : INFO : Ready to measure, file location: d:\data\2017-12\2017-12-30\2017-12-30_AWGGateSweep-#0003...


Instrument alazar_ctl already exists.


delay is less than recommended for filter choice: (expect delay >= 1e-06)
2017-12-30 20:35:06,545 : measurement : INFO : Samples per record: 8192


Instrument alazar_ctl already exists.


delay is less than recommended for filter choice: (expect delay >= 1e-06)
2017-12-30 20:35:07,086 : measurement : INFO : Updated integration time: 8.0875e-05
2017-12-30 20:35:07,087 : measurement : INFO : Updated samples per record: 8192
2017-12-30 20:35:07,088 : measurement : INFO : Time per gate voltage: 9e-05 s
2017-12-30 20:35:07,089 : measurement : INFO : Time per AWG sweep: 0.01802 s


Instrument alazar_ctl already exists.


delay is less than recommended for filter choice: (expect delay >= 1e-06)
C:\Users\Administrator\Code\Qcodes\qcodes\instrument\parameter.py:1097: UserWarning: MultiParameters do not support set at this time.
  warnings.warn('MultiParameters do not support set at this time.')
2017-12-30 20:37:53,804 : measurement : INFO : Measurement finished, cleaning up...
2017-12-30 20:37:53,894 : measurement : INFO : All done!
